In [ ]:
import uuid
import re
import json
import requests
from bs4 import BeautifulSoup

In [ ]:
# получаем html код страницы, если она доступна, иначе Error
def get_html_code(url: str) -> str:
    headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    try:
        req = requests.get(url, headers=headers)
        # The HTTP 200 OK success status response code indicates that the request has succeeded.
        if req.status_code == 200:
            return req.text
        else:
            return "Error"
        
    except Exception as e:
        print(e)
        return "Error"

# переводим html код в объект soup класса BeautifulSoup
def html_to_soup(html: str) -> BeautifulSoup:
    soup = BeautifulSoup(html, "html.parser")
    return soup

# вытаскивает из текста совпадения, используя регулярные выражения
def find_by_regex(text: str, regular_expression: re.Pattern) -> list[str]:
    matches = re.findall(regular_expression, text)
    matches = list(set(matches))
    return matches

# обогащает CVE описанием с mitre.org
def enrich_cve(cve: str) -> str:
    try:
        mitre_url = "https://cve.mitre.org/cgi-bin/cvename.cgi?name="
        cve_url = mitre_url + cve

        html_code = get_html_code(cve_url)
        soup = html_to_soup(html_code)

        description = soup.find("th", string="Description")
        description = description.find_next("td", colspan="2").text
    except Exception as e:
        print(e)
        return "Error"

    return description

# записывает данные в json файл
def write_to_json(filename: str, data: list[dict]):
    with open(filename, 'w') as f:
        json.dump(data,
                  f,
                  indent=4,
                  ensure_ascii=True)

# читает данные из json файла
def read_from_json(filename: str):
    with open(filename, 'r') as f:
        data = json.load(f)
    return data